## Preprocessing

### ✅ Preprocessing Checkpoints

* 1. Convert to lowercase
* 2. Remove punctuation
* 3. Remove stop words
* 4. Remove HTML tags, URLs, and brackets
* 5. Normalize chat words
* 6. Correct spelling
* 7. Skip emoji handling
* 8. Apply lemm

---


In [1]:
import string
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from bs4 import BeautifulSoup
from spellchecker import SpellChecker
import emoji

In [2]:
# ============================================================
# 1. Preview the reviews, find out the quality of the data
# ============================================================
df = pd.read_csv('IMDB Dataset.csv')
pd.set_option('display.max_colwidth', None)
df.head(20)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

#### Observation

| **Check**                        | **Issue Found?** | **Notes**                                                               |
| -------------------------------- | ---------------- | ----------------------------------------------------------------------- |
| **1. Lowercase and Capital**     | ✅ Yes            | Reviews mix lowercase and capitalized words; needs normalization.       |
| **2. Punctuations**              | ✅ Yes            | Many punctuations (`...`, `!!`, `??`, quotes, etc.) are used.           |
| **3. StopWords**                 | ✅ Yes            | Common stopwords like "is", "and", "the" are prevalent.                 |
| **4. HTML Tags, URLs, Brackets** | ✅ Yes            | Contains `<br />` tags, and some special characters like `<`, `>`.      |
| **5. ChatsWords**                | ⚠️ Few           | A few chat-like forms (e.g., "BTW", "mehehe", "wanna") appear sparsely. |
| **6. Misspelling**               | ✅ Yes            | Words like "descent" (intended “decent”), "them selfs", "scheisse".     |
| **7. Emoji**                     | ❌ No             | No emojis were found in the provided data, need to search further.                                    |


In [3]:
# ============================================================
# Check if there are any emojis in the reviews
# ============================================================
# Emoji Check
def extract_emojis(text):
    text = str(text)
    return ''.join(char for char in text if char in emoji.EMOJI_DATA)

# Apply the function
df['emojis_found'] = df['review'].apply(extract_emojis)

# Add a boolean flag whether emojis were found
df['has_emoji'] = df['emojis_found'].apply(lambda x: bool(x))

# Output sample
print(df[df['has_emoji']].head())

In [4]:
# ============================================================
"""
According to the observation, we need to do the following preprocessing:

# Cleaning
1. Lowercase
2. strip HTML
3. strip URLs
4. strip [bracketed] parts
5. remove emoji
6. strip punctuation
7. chat-word expansion
8. remove stop-words
9. spelling-correction block — takes time so commented out

# Reduce dimensionality
10. stemming or lemmatization(BETTER)

""" 
# ============================================================


chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()


# ============================================================
# Preprocessing pipeline
# ============================================================
def preprocessing_pipeline(review: str):
    """
    Clean and preprocess a review string using the full pipeline.

    Args:
        review (str): The raw review text.
    Returns:
        str: The cleaned and preprocessed review text.
    """
    # 1. quick lowercase
    review = review.lower()

    # 2. strip HTML
    review = BeautifulSoup(review, "html.parser").get_text()

    # 3. strip URLs
    review = re.sub(r'http\S+|www\.\S+', '', review, flags=re.IGNORECASE)

    # 4. strip [bracketed] parts
    review = re.sub(r'\[[^]]*\]', '', review)

    # 5. remove emoji
    review = emoji.replace_emoji(review, replace='')

    # 6. strip punctuation
    review = review.translate(str.maketrans('', '', string.punctuation))

    # 7. chat-word expansion  (done before stop-word removal)
    new_text = []
    for word in review.split():
        if word.upper() in chat_words:
            new_text.append(chat_words[word.upper()])
        else:
            new_text.append(word)
    review = " ".join(new_text)

    # 8. lowercase again so expansions are uniform
    review = review.lower()

    # 9. remove stop-words
    review = ' '.join([word for word in review.split() if word not in stop_words])

    # 10. spelling-correction block — takes time
    # spell = SpellChecker()
    # words = review.split()
    # corrected_words = []

    # for word in words:
    #     corrected = spell.correction(word)
    #     # If correction is None, use the original word
    #     corrected_words.append(corrected if corrected else word)

    # review = " ".join(corrected_words)


    # 11. stemming or lemmatization
    # review = ' '.join([ps.stem(word) for word in review.split()])
    review = ' '.join([lemmatizer.lemmatize(word) for word in review.split()])
    
    return review


In [7]:
# ============================================================
# Apply preprocessing pipeline to dataset, then check the results
# ============================================================
df_origin = pd.read_csv('IMDB Dataset.csv')
sample_reviews = df_origin['review'].sample(3, random_state=42)
processed_sample = sample_reviews.apply(lambda x: preprocessing_pipeline(x))

# Display the results
for original, processed in zip(sample_reviews, processed_sample):
    print("Original:\n", original[:300], "\n")
    print("Processed:\n", processed[:300], "\n")
    print("="*80)



Original:
 I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok  

Processed:
 really liked summerslam due look arena curtain look overall interesting reason anyways could one best summerslams ever wwf didnt lex luger main event yokozuna tear eye ok huge fat man v strong man im glad time changed terrible main event like every match luger terrible match card razor ramon v ted d 

Original:
 Not many television shows appeal to quite as many different kinds of fans like Farscape does...I know youngsters and 30/40+ years old;fans both Male and Female in as many different countries as you can think of that just adore this T.V miniseries. It has elements that can be found in almost every ot 

Processed:
 many television show appeal quite many differ

**Still has some misspelling issues, but the other issues have been solved.**